<a href="https://colab.research.google.com/github/asigalov61/Tegridy-MIDI-Dataset/blob/master/Advanced_MIDI_Silence_Removal_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced MIDI Silence Removal Tool (Ver. 1.9)

***

### Based on absolutely incredible code-repo-project by Brian Model: https://github.com/brianmodel/DeepOrchestration

***

#### Project Los Angeles

#### Tegridy Code 2020

***

In [ ]:
#@title Install dependencies
!pip install mido

In [ ]:
#@title Import modules and functions. Create IO dirs.
# Choose names for Source and Destination folders for conversion
source_dir = "/content/S" #@param {type:"string"}
dest_dir = "/content/D" #@param {type:"string"}
source_path = source_dir
dest_path = dest_dir

import os, glob
if not os.path.exists(source_path):
    os.makedirs(source_path)
if not os.path.exists(dest_path):
    os.makedirs(dest_path)


import numpy as np
import music21
from music21.interval import Interval
from music21.pitch import Pitch
import mido
from mido import MidiFile
import numpy as np

import tqdm.auto

def get_pianoroll_time(pianoroll):
    T_pr_list = []
    for k, v in pianoroll.items():
        T_pr_list.append(v.shape[0])
    if not len(set(T_pr_list)) == 1:
        print("Inconsistent dimensions in the new PR")
        return None
    return T_pr_list[0]


def get_pitch_dim(pianoroll):
    N_pr_list = []
    for k, v in pianoroll.items():
        N_pr_list.append(v.shape[1])
    if not len(set(N_pr_list)) == 1:
        print("Inconsistent dimensions in the new PR")
        raise NameError("Pr dimension")
    return N_pr_list[0]


def sum_along_instru_dim(pianoroll):
    T_pr = get_pianoroll_time(pianoroll)
    N_pr = get_pitch_dim(pianoroll)
    rp = np.zeros((T_pr, N_pr), dtype=np.int16)
    for k, v in pianoroll.items():
        rp = np.maximum(rp, v)
    return rp


def get_first_last_non_zero(pianoroll):
    PR = sum_along_instru_dim(pianoroll)
    first = min(np.nonzero(np.sum(PR, axis=1))[0])
    last = max(np.nonzero(np.sum(PR, axis=1))[0])
    return first, last


def clip_pr(pianoroll):
    # Remove zero at the beginning and end of prs
    out = {}
    start_time, end_time = get_first_last_non_zero(pianoroll)
    for k, v in pianoroll.items():
        out[k] = v[start_time:end_time, :]
    return out


def pitch_class(pr):
    nb_class = 12
    nb_pitch = pr.shape[1]
    if not nb_pitch == 128:
        raise Exception('Pitch dimension should be equal to 128')
    pr_red = np.zeros((pr.shape[0], nb_class))
    for p in range(nb_pitch):
        c = p % nb_class
        pr_red[:, c] += pr[:, p]
    # Binary
    pr_red_bin = (pr_red > 0).astype(int)
    return pr_red_bin

def extract_pianoroll_part(pianoroll, start_time, end_time):
    new_pr = {}
    # Start and end time are given in discrete frames
    for k,v in pianoroll.items():
        new_pr[k] = v[start_time:end_time]
    return new_pr

def remove_unused_pitch(pr, mapping):
    mapping_reduced = {}
    start_index = 0
    # First build the mapping
    for instru, pitch_range in mapping.iteritems():
        # Extract the pianoroll of this instrument
        start_index_pr = pitch_range[0]
        end_index_pr = pitch_range[1]
        pr_instru = pr[:, start_index_pr:end_index_pr]
        # Get lowest/highest used pitch
        existing_pitches = np.nonzero(pr_instru)[1]
        if np.shape(existing_pitches)[0] > 0:
            start_pitch = np.amin(existing_pitches)
            end_pitch = np.amax(existing_pitches) + 1
            end_index = start_index + end_pitch - start_pitch
            mapping_reduced[instru] = {'start_pitch': start_pitch,
                                       'end_pitch': end_pitch,
                                       'start_index': start_index,
                                       'end_index': end_index,
                                       # Indexex in the full matrix for reconstruction
                                       'start_index_rec': start_index_pr,
                                       'end_index_rec': end_index_pr}
            if 'pr_reduced' not in locals():
                pr_reduced = pr_instru[:, start_pitch:end_pitch]
            else:
                pr_reduced = np.concatenate((pr_reduced, pr_instru[:, start_pitch:end_pitch]), axis=1)

            start_index = end_index
    return pr_reduced, mapping_reduced


def remove_unmatched_silence(pr0, pr1):
    # Detect problems
    flat_0 = sum_along_instru_dim(pr0).sum(axis=1)
    flat_1 = sum_along_instru_dim(pr1).sum(axis=1)
    ind_clean = np.where(np.logical_not((flat_0 == 0) ^ (flat_1 == 0)))[0]

    def keep_clean(pr, ind_clean):
        pr_out = {}
        for k, v in pr.iteritems():
            pr_out[k] = v[ind_clean]
        return pr_out

    pr0_clean = keep_clean(pr0, ind_clean)
    pr1_clean = keep_clean(pr1, ind_clean)
    duration = len(ind_clean)

    return pr0_clean, pr1_clean, duration


def remove_silence(pr):
    # Detect silences
    flat = sum_along_instru_dim(pr).sum(axis=1)
    ind_clean = np.where(np.logical_not(flat == 0))[0]

    def keep_clean(pr, ind_clean):
        pr_out = {}
        for k, v in pr.items():
            pr_out[k] = v[ind_clean]
        return pr_out
    pr_clean = keep_clean(pr, ind_clean)

    mapping = np.zeros((flat.shape[0]), dtype=np.int) - 1
    counter = 0
    for elem in ind_clean:
        mapping[elem] = counter
        counter += 1

    return pr_clean, mapping


def remove_match_silence(pr0, pr1):
    # Detect problems
    flat_0 = sum_along_instru_dim(pr0).sum(axis=1)
    flat_1 = sum_along_instru_dim(pr1).sum(axis=1)
    ind_clean = np.where(np.logical_not((flat_0 == 0) & (flat_1 == 0)))[0]

    def keep_clean(pr, ind_clean):
        pr_out = {}
        for k, v in pr.iteritems():
            pr_out[k] = v[ind_clean]
        return pr_out

    pr0_clean = keep_clean(pr0, ind_clean)
    pr1_clean = keep_clean(pr1, ind_clean)
    duration = len(ind_clean)

    return pr0_clean, pr1_clean, duration


def reconstruct_full_pr(pr_reduced, mapping_reduced):
    # Catch full size pr pitch dimension
    max_index = 0
    for value in mapping_reduced.itervalues():
        max_index = max(max_index, value['end_index_rec'])
    pitch_dimension = max_index
    time_dimension = pr_reduced.shape[0]
    pr = np.zeros((time_dimension, pitch_dimension))
    mapping = {}
    for instru_name, value in mapping_reduced.iteritems():
        mapping[instru_name] = (value['start_index_rec'], value['end_index_rec'])
        start_index = value['start_index_rec'] + value['start_pitch']
        end_index = value['start_index_rec'] + value['end_pitch']
        # Small verification
        if end_index > value['end_index_rec']:
            raise NameError('Conflicting indices during reconstruction')
        pr[:, start_index: end_index] = pr_reduced[:, value['start_index']: value['end_index']]
    return pr, mapping

In [ ]:
#@title Declare MIDI Patches/Program Change Names
MIDI_PATCHES = {
    "Piccolo": 73,
    "Flute": 74,
    "Alto-Flute": 74,
    "Soprano-Flute": 74,
    "Bass-Flute": 74,
    "Contrabass-Flute": 74,
    "Pan Flute": 76,
    "Recorder": 74,
    "Ocarina": 80,
    "Oboe": 69,
    "Oboe-dAmore": 69,
    "Oboe de Caccia": 69,
    "English-Horn": 70,
    "Heckelphone": 0,
    "Piccolo-Clarinet-Ab": 72,
    "Clarinet": 72,
    "Clarinet-Eb": 72,
    "Clarinet-Bb": 72,
    "Piccolo-Clarinet-D": 72,
    "Clarinet-C": 72,
    "Clarinet-A": 72,
    "Basset-Horn-F": 61,
    "Alto-Clarinet-Eb": 72,
    "Bass-Clarinet-Bb": 72,
    "Bass-Clarinet-A": 72,
    "Contra-Alto-Clarinet-Eb": 72,
    "Contrabass-Clarinet-Bb": 72,
    "Bassoon": 71,
    "Contrabassoon": 71,
    "Soprano-Sax": 65,
    "Alto-Sax": 66,
    "Tenor-Sax": 67,
    "Baritone-Sax": 68,
    "Bass-Sax": 68,
    "Contrabass-Sax": 68,
    "Horn": 61,
    "Harmonica": 23,
    "Piccolo-Trumpet-Bb": 57,
    "Piccolo-Trumpet-A": 57,
    "High-Trumpet-F": 57,
    "High-Trumpet-Eb": 57,
    "High-Trumpet-D": 57,
    "Cornet": 57,
    "Trumpet": 57,
    "Trumpet-C": 57,
    "Trumpet-Bb": 57,
    "Cornet-Bb": 57,
    "Alto-Trumpet-F": 57,
    "Bass-Trumpet-Eb": 57,
    "Bass-Trumpet-C": 57,
    "Bass-Trumpet-Bb": 57,
    "Clarion": 57,
    "Trombone": 58,
    "Alto-Trombone": 58,
    "Soprano-Trombone": 58,
    "Tenor-Trombone": 58,
    "Bass-Trombone": 58,
    "Contrabass-Trombone": 58,
    "Euphonium": 59,
    "Tuba": 59,
    "Bass-Tuba": 59,
    "Contrabass-Tuba": 59,
    "Flugelhorn": 0,
    "Piano": 1,
    "Celesta": 9,
    "Organ": 20,
    "Harpsichord": 7,
    "Accordion": 22,
    "Bandoneone": 22,
    "Harp": 47,
    "Guitar": 25,
    "Bandurria": 25,
    "Mandolin": 25,
    "Lute": 25,
    "Lyre": 25,
    "Violin": 40,
    "Violins": 40,
    "Viola": 42,
    "Violas": 42,
    "Viola de gamba": 42,
    "Viola de braccio": 42,
    "Violoncello": 43,
    "Violoncellos": 43,
    "Contrabass": 44,
    "Basso continuo": 44,
    "Bass drum": 35,
    "Glockenspiel": 10,
    "Xylophone": 14,
    "Vibraphone": 12,
    "Marimba": 13,
    "Maracas": 70,
    "Bass-Marimba": 13,
    "Tubular-Bells": 15,
    "Clave": 75,
    "Bombo": 0,
    "Hi-hat": 42,
    "Triangle": 81,
    "Ratchet": 0,
    "Drum": 0,
    "Snare drum": 38,
    "Steel drum": 115,
    "Tambourine": 54,
    "Tam tam": 54,
    "Timpani": 48,
    "Cymbal": 49,
    "Castanets": 0,
    "Percussion": 0,
    "Voice": 54,
    "Voice soprano": 54,
    "Voice mezzo": 54,
    "Voice alto": 54,
    "Voice contratenor": 54,
    "Voice tenor": 54,
    "Voice baritone": 54,
    "Voice bass": 54,
    "Ondes martenot": 0,
    "Unknown": 0,
}

In [ ]:
#@title Remove silence

#@title Run this to reduce/filter all files. Choose Extensions, Name Tag, MIDI Patch Instrument Name (refer to the list at the end of the colab), and drums/debug options.
input_files_extension = "*.mid" #@param ["*.mid", "*.midi", "*.kar", "*.*"]
output_extension = ".mid" #@param {type:"string"}
output_files_names_tag = "_fixed.mid" #@param {type:"string"}
desired_output_MIDI_instrument = "Violin" #@param {type:"string"}
debug = False #@param {type:"boolean"}

if not os.path.exists(dest_path):
    os.makedirs(dest_path)

# Create a list of paths and files. Select desired MIDI extension
source_files = glob.glob(os.path.join(source_path, input_files_extension))
if debug: print(source_files)

path = source_dir

def transpose(file):
    score = music21.converter.parse(file)
    key = score.analyze("key")

    if key.mode == "minor":
        i = Interval(key.tonic, Pitch("A"))
    else:
        i = Interval(key.tonic, Pitch("C"))

    transposed = score.transpose(i)
    for part in transposed.parts:
         if debug: print(part)
    transposed = transposed.chordify()
    if debug:  print(transposed.__dict__)
    for element in transposed.notesAndRests:
         if debug:  print(element)
    return transposed

def stream_to_pr(stream):
    pr = np.empty((0, 128))
    for element in stream.notes:
        quant = np.zeros((1, 128))
        if isinstance(element, music21.chord.Chord):
            for note in element:
                quant[0][note_to_index(str(note.pitch))] = 1
        else:
            quant[0][note_to_index(str(element.pitch))] = 1
        if len(pr) == 0 or np.count_nonzero(quant) != 0 and (pr[-1] != quant).any():
            pr = np.append(pr, quant, axis=0)
    return pr

def note_to_index(note):
    mapping = {"C": 0, "D": 2, "E": 4, "F": 5, "G": 7, "A": 9, "B": 11}
    base = mapping[note[0]]
    scalar = int(note[-1])
    index = (scalar - 1) * 12 + base + 24
    if len(note) == 3:
        if note[1] == "-":
            index -= 1
        elif note[1] == "#":
            index += 1
    return index

#@title Write result back to MIDI file
output_MIDI_beats_per_measure = 4 #@param {type:"slider", min:1, max:16, step:1}
output_MIDI_tempo_rate = 80 #@param {type:"slider", min:0, max:200, step:20}

# Piano roll (what we create), quantization and path
def write_midi(pr, ticks_per_beat, write_path, tempo=80):
    def pr_to_list(pr):
        # List event = (pitch, velocity, time)
        T, N = pr.shape
        t_last = 0
        pr_tm1 = np.zeros(N)
        list_event = []
        for t in range(T):
            pr_t = pr[t]
            mask = pr_t != pr_tm1
            if (mask).any():
                for n in range(N):
                    if mask[n]:
                        pitch = n
                        velocity = int(pr_t[n])
                        # Time is incremented since last event
                        t_event = t - t_last
                        t_last = t
                        list_event.append((pitch, velocity, t_event))
            pr_tm1 = pr_t
        return list_event

    # Tempo
    microseconds_per_beat = mido.bpm2tempo(tempo)
    # Write a pianoroll in a midi file
    mid = MidiFile()
    mid.ticks_per_beat = ticks_per_beat

    # Each instrument is a track
    for instrument_name, matrix in pr.items():
        # Add a new track with the instrument name to the midi file
        track = mid.add_track(instrument_name)
        # transform the matrix in a list of (pitch, velocity, time)
        events = pr_to_list(matrix)
        # Tempo
        track.append(mido.MetaMessage("set_tempo", tempo=microseconds_per_beat))
        # Add the program_change
        try:
            program = MIDI_PATCHES[desired_output_MIDI_instrument]
        except:
            # Defaul is piano
            # print instrument_name + " not in the program_change mapping"
            # print "Default value is 0 (piano)"
            # print "Check acidano/data_processing/utils/program_change_mapping.py"
            program = 0
        track.append(mido.Message("program_change", program=program))

        # This list is required to shut down
        # notes that are on, intensity modified, then off only 1 time
        # Example :
        # (60,20,0)
        # (60,40,10)
        # (60,0,15)
        notes_on_list = []
        # Write events in the midi file
        for event in events:
            pitch, velocity, time = event
            if velocity == 0:
                # Get the channel
                track.append(
                    mido.Message("note_off", note=pitch, velocity=0, time=time)
                )
                notes_on_list.remove(pitch)
            else:
                if pitch in notes_on_list:
                    track.append(
                        mido.Message("note_off", note=pitch, velocity=0, time=time)
                    )
                    notes_on_list.remove(pitch)
                    time = 0

                rng = np.random.default_rng()
                sim_velocity = 64 + rng.integers(9)   
                track.append(
                    mido.Message("note_on", note=pitch, velocity=sim_velocity, time=time)
                )
                if debug: print(sim_velocity)
                notes_on_list.append(pitch)
    mid.save(write_path)
    return

print('Starting up...')
print('Removing Silence :)', desired_output_MIDI_instrument)
for fname in tqdm.auto.tqdm(source_files):
       if debug: print("Reading:", fname)
       try:
          midi = transpose(fname)
          midi = midi.chordify()
          for el in midi.notes:
            if debug: print(el)
          ypr = stream_to_pr(midi)
          if debug: print(ypr)
          remove_silence({desired_output_MIDI_instrument: ypr})
          outname = fname.replace(output_extension, output_files_names_tag).replace(source_path, dest_path)
          if debug: print(outname)
          #outmid.write(outname)
          write_midi({desired_output_MIDI_instrument: ypr}, output_MIDI_beats_per_measure, outname, output_MIDI_tempo_rate)  
       except KeyboardInterrupt:
           break
       except:
         continue